<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/7-sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk

#Partes de este notebook (RNN) fueron adaptadas desde el curso 
#fast.ai NLP course, USF programa MS Data Science May-June 2019.
#https://github.com/fastai/course-nlp

#El ejemplo de LSTM para el problema de POST fue adaptado desde
#https://www.kaggle.com/krishanudb/lstm-character-word-pos-tag-model-pytorch



In [ ]:
nltk.download("cess_esp")
nltk.download('universal_tagset')

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
#esto es un parche pues las etiquetas de cess_esp son antiguas y su mapeo a las 
#etiquetas universales no funciona bien
if nltk.corpus.cess_esp._tagset != "es-cast3lb":
  nltk.corpus.cess_esp._tagset = "es-cast3lb"
  nltk.tag.mapping._load_universal_map("es-cast3lb")  # initialize; normally loaded on demand
  mapdict = nltk.tag.mapping._MAPPINGS["es-cast3lb"]["universal"] # shortcut to the map
  alltags = set(t for w, t in nltk.corpus.cess_esp.tagged_words())
  for tag in alltags:
      if len(tag) <= 2:   # These are complete
          continue
      mapdict[tag] = mapdict[tag[:2]]

In [ ]:
es_pos_data = nltk.corpus.cess_esp.tagged_sents(tagset='universal')

In [ ]:
es_pos_data

[[('El', 'DET'), ('grupo', 'NOUN'), ('estatal', 'ADJ'), ('Electricité_de_France', 'NOUN'), ('-Fpa-', '.'), ('EDF', 'NOUN'), ('-Fpt-', '.'), ('anunció', 'VERB'), ('hoy', 'ADV'), (',', '.'), ('jueves', 'X'), (',', '.'), ('la', 'DET'), ('compra', 'NOUN'), ('del', 'ADP'), ('51_por_ciento', 'NUM'), ('de', 'ADP'), ('la', 'DET'), ('empresa', 'NOUN'), ('mexicana', 'ADJ'), ('Electricidad_Águila_de_Altamira', 'NOUN'), ('-Fpa-', '.'), ('EAA', 'NOUN'), ('-Fpt-', '.'), (',', '.'), ('creada', 'ADJ'), ('por', 'ADP'), ('el', 'DET'), ('japonés', 'ADJ'), ('Mitsubishi_Corporation', 'NOUN'), ('para', 'ADP'), ('poner_en_marcha', 'VERB'), ('una', 'DET'), ('central', 'NOUN'), ('de', 'ADP'), ('gas', 'NOUN'), ('de', 'ADP'), ('495', 'X'), ('megavatios', 'NOUN'), ('.', '.')], [('Una', 'DET'), ('portavoz', 'NOUN'), ('de', 'ADP'), ('EDF', 'NOUN'), ('explicó', 'VERB'), ('a', 'ADP'), ('EFE', 'NOUN'), ('que', 'CONJ'), ('el', 'DET'), ('proyecto', 'NOUN'), ('para', 'ADP'), ('la', 'DET'), ('construcción', 'NOUN'), ('de'

## Actividad 1: HMM

Tipos de palabras:


*   VERB - verbos
*   NOUN - sustantivo
*   PRON - pronombre 
*   ADJ - adjetivo
*   ADV - adverbio
*   ADP - preposición o posposición
*   CONJ - conjunción
*   DET - determinantes
*   NUM - número
*   PRT - partícula u otra palabra funcionale
*   X - otro
*   . - puntuación



In [ ]:
hmm_trainer = nltk.tag.hmm.HiddenMarkovModelTrainer()

In [ ]:
tagger = hmm_trainer.train_supervised(es_pos_data)

In [ ]:
print( tagger.tag("No coma ahora que entró en un coma inducido.".split()) )

[('No', 'ADV'), ('coma', 'NOUN'), ('ahora', 'ADV'), ('que', 'PRON'), ('entró', 'VERB'), ('en', 'ADP'), ('un', 'DET'), ('coma', 'NOUN'), ('inducido.', 'DET')]


In [ ]:
print( tagger.tag("El perro de Juan saltó muy alto.".split()) )

[('El', 'DET'), ('perro', 'NOUN'), ('de', 'ADP'), ('Juan', 'NOUN'), ('saltó', 'VERB'), ('muy', 'ADV'), ('alto.', 'DET')]


¿Cuáles son los símbolos de este modelo de HMM? ¿cuántos estados tiene? ¿qué  algoritmo se esta usando para crear el HMM?

In [ ]:
# programe y responda
#
#
#

# Redes neuronales



In [ ]:
import numpy
from sklearn.datasets import make_blobs
from fastai.text import *
import fastai

## Ejemplo de uso de pytorch Modelo totalmente conectado

In [ ]:
## SOLUTION START
class Feedforward(nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output
# CREATE RANDOM DATA POINTS

def blob_label(y, label, loc): # assign labels
    target = numpy.copy(y)
    for l in loc:
        target[y == l] = label
    return target

x_train, y_train = make_blobs(n_samples=40, n_features=2, cluster_std=1.5, shuffle=True)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(blob_label(y_train, 0, [0]))
y_train = torch.FloatTensor(blob_label(y_train, 1, [1,2,3]))

x_test, y_test = make_blobs(n_samples=10, n_features=2, cluster_std=1.5, shuffle=True)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(blob_label(y_test, 0, [0]))
y_test = torch.FloatTensor(blob_label(y_test, 1, [1,2,3]))

model = Feedforward(2, 10)
#ENTRENAMIENTO MANUAL
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())
model.train()
epoch = 20
for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())
#SOLUTION END

Test loss before training 0.44255709648132324
Epoch 0: train loss: 0.7305599451065063
Epoch 1: train loss: 0.7124975323677063
Epoch 2: train loss: 0.6969451904296875
Epoch 3: train loss: 0.6835895776748657
Epoch 4: train loss: 0.6721459627151489
Epoch 5: train loss: 0.6623579859733582
Epoch 6: train loss: 0.6539974808692932
Epoch 7: train loss: 0.6468626856803894
Epoch 8: train loss: 0.6407763361930847
Epoch 9: train loss: 0.6355843544006348
Epoch 10: train loss: 0.6311529874801636
Epoch 11: train loss: 0.6273666620254517
Epoch 12: train loss: 0.6241262555122375
Epoch 13: train loss: 0.6213471293449402
Epoch 14: train loss: 0.6189568638801575
Epoch 15: train loss: 0.6168938875198364
Epoch 16: train loss: 0.6151062250137329
Epoch 17: train loss: 0.6135494709014893
Epoch 18: train loss: 0.6121864318847656
Epoch 19: train loss: 0.6109856367111206
Test loss after Training 0.42662397027015686


## Redes recurrentes

### Cargando datos para pytorch

In [ ]:
# from fastai.text import *
# import fastai

#fastai.torch_core.defaults.device = torch.device('cuda',0)
fastai.torch_core.defaults.device = torch.device('cpu')
defaults.device

bs=64
path = untar_data(URLs.HUMAN_NUMBERS)
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]
train_txt = readnums('train.txt'); train_txt[0][:80]
valid_txt = readnums('valid.txt'); valid_txt[0][-80:]
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)
src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs, bptt=3)
v = data.valid_ds.vocab

#miremos los datos
print(valid_txt[0][:80])

#numero de tokens en total
print(len(data.train_ds[0][0].data))


print(v.itos)

eight thousand one, eight thousand two, eight thousand three, eight thousand fou
50079
['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxmaj', 'xxup', 'xxrep', 'xxwrep', ',', 'hundred', 'thousand', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'xxfake']


In [ ]:
#revisemos las dimensiones
print( data.bptt) #esto es una division para el entrenamiento de RNN
print(len(data.train_dl))

it = iter(data.train_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

print(x1.numel()+x2.numel()+x3.numel())

print(x1.shape)

x1, y1

3
261
576
torch.Size([64, 3])


(tensor([[13,  9, 14],
         [13, 10, 30],
         [10, 26, 17],
         [ 9, 16, 10],
         [18, 10, 20],
         [ 9, 19, 10],
         [11, 23, 13],
         [11, 12, 10],
         [13, 10, 28],
         [19,  9, 12],
         [16, 10, 21],
         [23, 13,  9],
         [24, 19,  9],
         [ 9, 12, 11],
         [12, 11, 20],
         [ 9, 13, 11],
         [13, 11, 13],
         [14, 10, 26],
         [15, 10, 27],
         [11, 17, 10],
         [10, 21, 13],
         [22, 19,  9],
         [15,  9, 13],
         [ 9, 14, 11],
         [10, 37,  9],
         [14, 11, 14],
         [11, 15, 10],
         [16, 10, 25],
         [10, 27, 14],
         [28, 20,  9],
         [11, 20, 10],
         [ 9, 15, 11],
         [20,  9, 15],
         [ 9, 15, 11],
         [15, 11, 15],
         [15, 11, 16],
         [11, 17, 10],
         [18, 10, 25],
         [10, 26, 20],
         [28, 16,  9],
         [23, 14,  9],
         [24, 20,  9],
         [ 9, 16, 11],
         [1

In [ ]:
#miremos algunos vectores
print(x1[0])
print(y1[0])
print(v.textify(x1[0]))
print(v.textify(y1[0]))

tensor([13,  9, 14])
tensor([ 9, 14,  9])
two , three
, three ,


In [ ]:
nh=64
bs=64

data = src.databunch(bs=bs, bptt=3)
nv = len(v.itos)
print(nv)


x,y = data.one_batch()
print(x.shape)
print(y.shape)

print(x)
print(y)

40
torch.Size([64, 3])
torch.Size([64, 3])
tensor([[13,  9, 14],
        [13, 10, 30],
        [10, 26, 17],
        [ 9, 16, 10],
        [18, 10, 20],
        [ 9, 19, 10],
        [11, 23, 13],
        [11, 12, 10],
        [13, 10, 28],
        [19,  9, 12],
        [16, 10, 21],
        [23, 13,  9],
        [24, 19,  9],
        [ 9, 12, 11],
        [12, 11, 20],
        [ 9, 13, 11],
        [13, 11, 13],
        [14, 10, 26],
        [15, 10, 27],
        [11, 17, 10],
        [10, 21, 13],
        [22, 19,  9],
        [15,  9, 13],
        [ 9, 14, 11],
        [10, 37,  9],
        [14, 11, 14],
        [11, 15, 10],
        [16, 10, 25],
        [10, 27, 14],
        [28, 20,  9],
        [11, 20, 10],
        [ 9, 15, 11],
        [20,  9, 15],
        [ 9, 15, 11],
        [15, 11, 15],
        [15, 11, 16],
        [11, 17, 10],
        [18, 10, 25],
        [10, 26, 20],
        [28, 16,  9],
        [23, 14,  9],
        [24, 20,  9],
        [ 9, 16, 11],
        [16

## Un primer ejemplo (Model 0) con la red desenrollada

Layer names:
- `i_h`: input to hidden
- `h_h`: hidden to hidden
- `h_o`: hidden to output
- `bn`: batchnorm

Como primer ejemplo se usara ultima palabra como salida. Esto requiere redefinir las funciones error

In [ ]:
def loss4(input,target): 
  return F.cross_entropy(input, target[:,-1])
def acc4 (input,target):
  #print('-------')
  #print(input.shape)
  #print(target.shape)

  #print(input)
  #print(target[:,-1])
  #print(accuracy(input, target[:,-1]))
  
  return accuracy(input, target[:,-1])

In [ ]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        #print(x.shape[0])#  -> 64
        #print(x.shape[1])#  -> 3 
        h = self.bn(F.relu(self.i_h(x[:,0])))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))

        #print(h)
        return self.h_o(h)

In [ ]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.679572,3.562212,0.076057,00:01
1,2.816533,2.832781,0.430147,00:01
2,2.137094,2.355204,0.457261,00:01
3,1.889885,2.186115,0.460938,00:01
4,1.808680,2.135613,0.461857,00:01
5,1.792184,2.128844,0.462546,00:01


## Un segundo ejemplo (Model 1) con la red desenrollada pero usando un ciclo



In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)


learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.511042,3.450875,0.119485,00:01
1,2.567659,2.525012,0.449449,00:01
2,2.053900,2.137320,0.460708,00:01
3,1.860536,2.011841,0.465533,00:01
4,1.791066,1.974570,0.467831,00:01
5,1.776352,1.969590,0.467831,00:01


## Un tercer ejemplo (Model 2) pero ahora no solo predecir la ultima palabra sino predecir con la 1 la palabra 2, y con 1 y la 2 la tercera, para utilizar mejor los datos.

In [ ]:
data = src.databunch(bs=bs, bptt=20)
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        #print(x.shape[0])  -> 64
        #print(x.shape[1])  -> 20
         
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
            
        return torch.stack(res, dim=1)

In [ ]:
learn = Learner(data, Model2(), metrics=accuracy)
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.781072,3.838687,0.041335,00:01
1,3.609435,3.662476,0.096449,00:01
2,3.403097,3.490098,0.182955,00:01
3,3.196812,3.337445,0.215128,00:01
4,3.012440,3.212076,0.228480,00:01
5,2.862563,3.119693,0.254901,00:01
6,2.750993,3.061484,0.265980,00:01
7,2.675124,3.029965,0.269460,00:01
8,2.629149,3.017810,0.271236,00:01
9,2.605283,3.015990,0.271591,00:01


Notar que el rendimiento bajo! (hay menos historia para predecir)

## Un cuarto ejemplo (Model 3), mejora levemente el rendimiento, ¿a qué se debe la diferencia?

In [ ]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh)
        
    def forward(self, x):
        res = []
        #si quieren probar en GPU!
        #h = self.h.to('cuda')
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [ ]:
model3=Model3()
#model3.cuda()
model3.cpu()
#al construir un "DataBunch" puedo entrenar directamente
learn = Learner(data, model3, metrics=accuracy)
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.556118,3.389250,0.155895,00:01
1,2.976181,2.428564,0.465909,00:00
2,2.250184,2.067140,0.323935,00:00
3,1.832530,2.164774,0.317685,00:01
4,1.627697,2.164879,0.320099,00:00
5,1.502320,2.015752,0.380753,00:00
6,1.368947,2.296233,0.363565,00:00
7,1.223000,2.309651,0.371733,00:00
8,1.099419,2.213777,0.379616,00:01
9,0.999777,2.084872,0.403196,00:01


Ahora usamos directamente una RNN

In [ ]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(data, Model4(), metrics=accuracy)
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.439745,3.130227,0.356463,00:00
1,2.754354,2.151671,0.464844,00:00
2,2.119184,2.002382,0.394318,00:00
3,1.769323,2.036179,0.315696,00:00
4,1.575828,1.921741,0.443537,00:00
5,1.439355,1.907566,0.490980,00:00
6,1.273818,1.829573,0.511364,00:00
7,1.102830,1.729319,0.521520,00:00
8,0.942475,1.599891,0.538992,00:00
9,0.795896,1.414066,0.576278,00:00


Ahora un modulo GRU en como RNN

In [ ]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(data, Model5(), metrics=accuracy)
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.708057,2.210277,0.440341,00:01
1,1.596552,1.346402,0.647656,00:01
2,0.781611,0.921727,0.766477,00:01
3,0.379948,0.851926,0.813920,00:01
4,0.191776,0.883512,0.820383,00:01
5,0.102014,1.039995,0.787500,00:01
6,0.057722,0.995301,0.795242,00:01
7,0.034585,1.142533,0.788920,00:01
8,0.022576,1.150154,0.793253,00:01
9,0.016299,1.150490,0.790838,00:01


## POS tagger en español con LSTM

In [ ]:
def prepare_sequence(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    # idxs = [to_ix[w] for w in seq]
    idxs = []
    for w in seq:
      if w in to_ix:
        idxs.append(to_ix[w])
      else:
        idxs.append(to_ix["xxunk"])
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
word_to_ix = {"xxunk":0}
tag_to_ix = {}
for sent in es_pos_data:
  for token in sent:
    word = token[0]
    tag = token[1]
    if word not in word_to_ix.keys():
      word_to_ix[word] = len(word_to_ix)
    if tag not in tag_to_ix.keys():
      tag_to_ix[tag] = len(tag_to_ix)

In [ ]:
EMBEDDING_DIM = 64
HIDDEN_DIM = 64

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
# Here I initialize the model with all the necesarry parameters
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix.keys()), len(tag_to_ix.keys()))

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    model.cuda()

# Define the loss function as the Negative Log Likelihood loss (NLLLoss)
loss_function = nn.NLLLoss()

# We will be using a simple SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

# The test sentence
seq1 = "No coma ahora que entró en un coma inducido .".split()
seq2 = "El perro de Juan saltó muy alto .".split()
print("Running a check on the model before training.\nSentences:\n{}\n{}".format(" ".join(seq1), " ".join(seq2)))
with torch.no_grad():
    for seq in [seq1, seq2]:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs.to(device))
        _, indices = torch.max(tag_scores, 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)
    
print("Training Started")
for epoch in range(10):
    for sentence in es_pos_data[:10000]:
        sentence_in = [token[0] for token in sentence]
        tags = [token[1] for token in sentence]
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in.to(device))
        
        loss = loss_function(tag_scores.to(device), targets.to(device))
        loss.backward()
        optimizer.step()
    print("epoch finished")
print("Training Finished!!!\nAgain testing on unknown data")
with torch.no_grad():
    for seq in [seq1, seq2]:
        inputs = prepare_sequence(seq, word_to_ix)
        tag_scores = model(inputs.to(device))
        _, indices = torch.max(tag_scores.to(device), 1)
        ret = []
        for i in range(len(indices)):
            for key, value in tag_to_ix.items():
                if indices[i] == value:
                    ret.append((seq[i], key))
        print(ret)

Running a check on the model before training.
Sentences:
No coma ahora que entró en un coma inducido .
El perro de Juan saltó muy alto .
[('No', 'CONJ'), ('coma', 'ADJ'), ('ahora', 'CONJ'), ('que', 'ADJ'), ('entró', 'ADJ'), ('en', 'NUM'), ('un', 'ADJ'), ('coma', 'ADJ'), ('inducido', 'ADJ'), ('.', 'ADJ')]
[('El', 'ADJ'), ('perro', 'VERB'), ('de', 'CONJ'), ('Juan', 'ADJ'), ('saltó', 'PRON'), ('muy', 'PRON'), ('alto', 'CONJ'), ('.', 'CONJ')]
Training Started
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
epoch finished
Training Finished!!!
Again testing on unknown data
[('No', 'ADP'), ('coma', 'ADP'), ('ahora', 'DET'), ('que', 'DET'), ('entró', 'DET'), ('en', 'ADP'), ('un', 'DET'), ('coma', 'DET'), ('inducido', 'DET'), ('.', 'ADP')]
[('El', 'ADP'), ('perro', 'ADP'), ('de', 'DET'), ('Juan', 'DET'), ('saltó', 'DET'), ('muy', 'ADP'), ('alto', 'DET'), ('.', 'ADP')]
